In [25]:
import numpy as np
import pandas 
from os import listdir
from os.path import isfile, join

In [26]:
gamer = [4, 5]
non_gamer = [1, 2, 3]

In [27]:
data_files_path = "./data/"
information_file_name = "data.csv"
data_files = [f for f in listdir(data_files_path) if isfile(join(data_files_path, f))]

In [28]:
attention_test_files = [file for file in data_files if file.lower().startswith("attention")]
n_back_test_files = [file for file in data_files if file.lower().startswith("n")]
stroop_test_files = [file for file in data_files if file.lower().startswith("stroop")]
print("Attention test file count: {}".format(len(attention_test_files)))
print("N-Back test file count: {}".format(len(n_back_test_files)))
print("Stroop test file count: {}".format(len(stroop_test_files)))

Attention test file count: 231
N-Back test file count: 241
Stroop test file count: 268


In [29]:
file = pandas.read_csv(information_file_name, delimiter=";")
file.columns

Index(['participant', 'Gracz_czy_niegracz:1', 'Typologia1:1', 'Typologia1:2',
       'Typologia1:3', 'Typologia1:4', 'Typologia1:5', 'Typologia1:6',
       'Typologia1:7', 'Typologia1:8', 'Typologia1:9', 'Typologia1:10',
       'Typologia1:11', 'Typologia1:12', 'Typologia1:13', 'Typologia1:14',
       'Typologia1:15', 'Typologia1:16', 'Typologia1:17', 'Typologia1:18',
       'experyment1:1', 'experyment2:1', 'experyment3:1', 'TIME_start',
       'TIME_end', 'TIME_total'],
      dtype='object')

In [30]:
#Filtrowanie rekordów bez wszystkich testów i bez typologii. TLDR mają mieć typologię lub wszystkie testy
filtered_file = file[\
    ((~file["experyment1:1"].isnull()) & (~file["experyment2:1"].isnull()) & (~file["experyment3:1"].isnull()) \
    |  (~file["Typologia1:1"].isnull()))]
print("Number of correct records: {}".format(len(filtered_file)))

players_type_data = pandas.read_csv(data_files_path + "players_types_data.csv", delimiter=";", encoding = 'unicode_escape')
print("Number of rows in players_types_data.csv: {}".format(len(players_type_data)))

Number of correct records: 284
Number of rows in players_types_data.csv: 284


In [31]:
players_type_data.columns

Index(['participant', 'grupa', 'Plec:1', 'Wiek:1', 'Gracz_czy_niegracz:1',
       'Gatunek_gry:1', 'Typologia1:1', 'Typologia1:2', 'Typologia1:3',
       'Typologia1:4', 'Typologia1:5', 'Typologia1:6', 'Typologia1:7',
       'Typologia1:8', 'Typologia1:9', 'Typologia1:10', 'Typologia1:11',
       'Typologia1:12', 'Typologia1:13', 'Typologia1:14', 'Typologia1:15',
       'Typologia1:16', 'Typologia1:17', 'Typologia1:18', 'SPO£ECZNIK',
       'ODKRYWCA', 'ZDOBYWCA', 'ZABÓJCA', 'TYP GRACZA'],
      dtype='object')

In [32]:
class Player:
    def __init__(self, id, if_gamer, stroop_test_filename, n_back_test_filename, attention_test_filename):
        self.test_result = {}

        self.test_result["id"] = id
        self.test_result["gender"] = None
        self.test_result["if_gamer"] = 1 if if_gamer else 0
        self.test_result["game_genres"] = None
        self.test_result["typology"] = []
        self.test_result["player_type_values"] = []
        self.test_result["player_type"] = ""
        self.test_result["stroop_reaction_prcnt"] = None
        self.test_result["stroop_reaction_time_avg"] = None
        self.test_result["stroop_reaction_compatible_prcnt"] = None
        self.test_result["stroop_reaction_time_compatible_avg"] = None
        self.test_result["stroop_reaction_incompatible_prcnt"] = None
        self.test_result["stroop_reaction_time_incompatible_avg"] = None
        self.test_result["n_back_reaction_avg"] = None
        self.test_result["n_back_reaction_time_avg"] = None  
        self.test_result["n_back_match_percentage"] = None 
        self.test_result["n_back_miss_percentage"] = None 
        self.test_result["n_back_false_alarm_percentage"] = None 
        self.test_result["attention_reaction_time_avg"] = [None]*5
        self.test_result["attention_status_prcnt"] = [None]*5       

        self.stroop_test_filename = stroop_test_filename
        self.n_back_test_filename = n_back_test_filename
        self.attention_test_filename = attention_test_filename 

        self.n_back_test_result()
        self.stroop_test_result()
        self.attention_test_result()

    def show_tests_results(self):
        print("Czy jest graczem: {}".format(self.test_result["if_gamer"]))
        print("Płeć: {}".format(self.test_result["gender"]))
        print("Typy gier: {}".format(self.test_result["game_genres"]))
        print("Identyfikator osoby: {}".format(self.test_result["id"]))
        print("Typologia osoby: {}".format(self.test_result["typology"]))
        print("Wartości dla poszczególnych typów: {}".format(self.test_result["player_type_values"]))
        print("Typ gracza: {}".format(self.test_result["player_type"]))
        if(self.test_result["n_back_reaction_time_avg"] != None):
            print("-*-"*20 + "\nn_back test:")
            print(("Średni adekwatność reakcji: {}\nŚredni czas reakcji: {}\n" \
                + "Procent dopasowanych: {}%\nProcent nietrafionych: {}%\nProcent fałszywie zgłoszonych: {}%")\
                .format(self.test_result["n_back_reaction_avg"], self.test_result["n_back_reaction_time_avg"],\
                     self.test_result["n_back_match_percentage"] * 100, self.test_result["n_back_miss_percentage"] * 100, self.test_result["n_back_false_alarm_percentage"] * 100))
        if(self.test_result["stroop_reaction_compatible_prcnt"] != None):
            print("-*-"*20 + "\nstroop test:")
            print(("Wszystkie próby:\n\tŚrednia adekwatność reakcji: {}\n\tŚredni Czas reakcji: {}\n" \
                + "Próby zgodne:\n\tŚrednia adekwatność reakcji: {}\n\tŚredni Czas reakcji: {}\n" \
                    + "Próby niezgodne:\n\tŚrednia adekwatność reakcji: {}\n\tŚredni czas reakcji: {}")\
                .format(\
            self.test_result["stroop_reaction_prcnt"] * 100, self.test_result["stroop_reaction_time_avg"],\
                self.test_result["stroop_reaction_compatible_prcnt"] * 100, self.test_result["stroop_reaction_time_compatible_avg"],\
                     self.test_result["stroop_reaction_incompatible_prcnt"] * 100, self.test_result["stroop_reaction_time_incompatible_avg"]))
        if(self.test_result["attention_reaction_time_avg"] != None):
            print("-*-"*20 + "\nattention test:")
            print([touple for touple in zip(self.test_result["attention_reaction_time_avg"],\
                 [prcnt * 100 for prcnt in self.test_result["attention_status_prcnt"]])])
            print(("Ogólny wynik:\n\tŚredni czas reakcji: {}\n\tŚredni rezultat reakcji: {}\n" \
                + "Wynik przy 5 dystraktorach:\n\tŚredni czas reakcji: {}\n\tŚredni rezultat reakcji: {}\n" \
                    + "Wynik przy 10 dystraktorach:\n\tŚredni czas reakcji: {}\n\tProcentowy rezultat reakcji: {}\n" \
                        + "Wynik przy 15 dystraktorach:\n\tŚredni czas reakcji: {}\n\tProcentowy rezultat reakcji: {}\n" \
                            + "Wynik przy 20 dystraktorach:\n\tŚredni czas reakcji: {}\n\tProcentowy rezultat reakcji: {}")\
                                .format(self.test_result["attention_reaction_time_avg"][0], self.test_result["attention_status_prcnt"][0] * 100,\
                    self.test_result["attention_reaction_time_avg"][1], self.test_result["attention_status_prcnt"][1] * 100,\
                    self.test_result["attention_reaction_time_avg"][2], self.test_result["attention_status_prcnt"][2] * 100,\
                    self.test_result["attention_reaction_time_avg"][3], self.test_result["attention_status_prcnt"][3] * 100,\
                    self.test_result["attention_reaction_time_avg"][4], self.test_result["attention_status_prcnt"][4] * 100))

    def get_results(self):
        return self.test_result

    def n_back_test_result(self):
        if type(self.n_back_test_filename) == str:
            results = pandas.read_csv(data_files_path + self.n_back_test_filename, delim_whitespace=True,\
                 names=["block_number", "trial_number", "trial_type", \
                     "score", "match", "miss", "false_alarm", "reaction_time", "memory", "current_letter", "nback1", "nback2"])
            
            self.test_result["n_back_reaction_avg"] = max(-1, results["score"].mean())
            self.test_result["n_back_match_percentage"] =\
                 len(results[results["match"] == 1])/(len(results[results["match"] == 0]) + len(results[results["match"] == 1]))
            self.test_result["n_back_miss_percentage"] =\
                 len(results[results["miss"] == 1])/(len(results[results["miss"] == 0]) + len(results[results["miss"] == 1]))
            self.test_result["n_back_false_alarm_percentage"] =\
                 len(results[results["false_alarm"] == 1])/(len(results[results["false_alarm"] == 0]) + len(results[results["false_alarm"] == 1]))
            self.test_result["n_back_reaction_time_avg"] = max(-1, results[results["match"] == 1]["reaction_time"].mean())
            
    def stroop_test_result(self):
        if type(self.stroop_test_filename) == str:
            results = pandas.read_csv(data_files_path + self.stroop_test_filename, delim_whitespace=True,\
                 names=["block_name", "word_name", "word_color", "color_match", "row_number", "pressed_key_number", "status", "response_time"])
            results = results[results["block_name"] != "training"]
            
            self.test_result["stroop_reaction_prcnt"] = len(results[results["status"].eq(1)])/len(results)
            self.test_result["stroop_reaction_time_avg"] = max(-1, results["response_time"].mean())
            self.test_result["stroop_reaction_compatible_prcnt"] = len(results[results["color_match"].eq(1) & results["status"].eq(1)])/\
                len(results[results["color_match"].eq(1)])
            self.test_result["stroop_reaction_incompatible_prcnt"] = len(results[results["color_match"].eq(0) & results["status"].eq(1)])/\
                len(results[results["color_match"].eq(0)])
            self.test_result["stroop_reaction_time_compatible_avg"] = max(-1, results[results["color_match"] == 1]["response_time"].mean())
            self.test_result["stroop_reaction_time_incompatible_avg"] = max(-1, results[results["color_match"] == 0]["response_time"].mean())

    def attention_test_result(self):
        if type(self.attention_test_filename) == str:
            results = pandas.read_csv(data_files_path + self.attention_test_filename, delim_whitespace=True,\
             names=["block_name", "displayed_number", "distraction", "number_of_distractions", "status", "response_time"])
            results = results[results["block_name"] != "training"]

            self.test_result["attention_reaction_time_avg"] = [max(-1, results[results["distraction"].eq(1) & results["status"].eq(1)]["response_time"].mean())]
            self.test_result["attention_status_prcnt"] = [len(results[(results["distraction"].eq(0) & results["status"].eq(3)) |\
                 (results["distraction"].eq(1) & results["status"].eq(1))])/\
                len(results)]
            for i in [5, 10, 15, 20]:
                self.test_result["attention_reaction_time_avg"].append(max(-1, results[results["number_of_distractions"].eq(i) & results["distraction"].eq(1) & results["status"].eq(1)]["response_time"].mean()))
                self.test_result["attention_status_prcnt"].append(len(results[results["number_of_distractions"].eq(i) & \
                ((results["distraction"].eq(0) & results["status"].eq(3)) | (results["distraction"].eq(1) & results["status"].eq(1)))])/\
                len(results[results["number_of_distractions"].eq(i)]))



In [33]:
#'SPO£ECZNIK','ODKRYWCA', 'ZDOBYWCA', 'ZABÓJCA', 'TYP GRACZA'
players = []
player_types_column_names = ["SPO£ECZNIK", 'ODKRYWCA', 'ZDOBYWCA', 'ZABÓJCA']
player_types_column_names_SV = ["SPOLECZNIK", 'ODKRYWCA', 'ZDOBYWCA', 'ZABOJCA']
typology_column_names = ['Typologia1:1', 'Typologia1:2',
       'Typologia1:3', 'Typologia1:4', 'Typologia1:5', 'Typologia1:6',
       'Typologia1:7', 'Typologia1:8', 'Typologia1:9', 'Typologia1:10',
       'Typologia1:11', 'Typologia1:12', 'Typologia1:13', 'Typologia1:14',
       'Typologia1:15', 'Typologia1:16', 'Typologia1:17', 'Typologia1:18']
for index, record in filtered_file.iterrows():
     players.append(Player(record['participant'], record['Gracz_czy_niegracz:1'] in gamer,\
         record['experyment1:1'], record['experyment2:1'], record['experyment3:1']))
     player_data = players_type_data[players_type_data["participant"] == players[-1].test_result["id"]]
     players[-1].test_result["gender"] = player_data["Plec:1"].values[0]
     players[-1].test_result["game_genres"] = player_data["Gatunek_gry:1"].values[0]
     if(players[-1].test_result["if_gamer"]):
          player_data_type = next(player_data[player_types_column_names].iterrows())[1]
          for id, value in enumerate(player_data_type):
               if (value == player_data_type.max()):
                    players[-1].test_result["player_type"] += str(id+1)+","
          if(players[-1].test_result["player_type"].endswith(",")):
               players[-1].test_result["player_type"] = players[-1].test_result["player_type"][:-1] 
     else:
          players[-1].test_result["player_type"] = None
     player_data_typology = next(player_data[typology_column_names].iterrows())[1]      
     for value in player_data_typology:
          if(players[-1].test_result["if_gamer"]):
               players[-1].test_result["typology"].append(value)
          else:
               players[-1].test_result["typology"].append(None)
     player_data_type_values = next(player_data[player_types_column_names].iterrows())[1]     
     for value in player_data_type_values:
          if(players[-1].test_result["if_gamer"]):
               players[-1].test_result["player_type_values"].append(value)
          else:
               players[-1].test_result["player_type_values"].append(None)
               

In [34]:
def show_player(id, players):
    for player in players:
        if player.test_result["id"] == id:
            player.show_tests_results()

In [35]:
#Prezentacja
show_player("s.58dd59e5-94b0-4818-b05e-694c267d30d8.txt", players)

Czy jest graczem: 1
Płeć: 2
Typy gier: Unturned, league of legends, civilization 6
Identyfikator osoby: s.58dd59e5-94b0-4818-b05e-694c267d30d8.txt
Typologia osoby: [1.0, 2.0, 6.0, 3.0, 3.0, 5.0, 3.0, 5.0, 3.0, 3.0, 5.0, 6.0, 1.0, 2.0, 2.0, 5.0, 2.0, 5.0]
Wartości dla poszczególnych typów: [9, 5, 6, 8]
Typ gracza: 1
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
n_back test:
Średni adekwatność reakcji: 1.0
Średni czas reakcji: 1014.6875
Procent dopasowanych: 26.666666666666668%
Procent nietrafionych: 0.0%
Procent fałszywie zgłoszonych: 0.0%
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
stroop test:
Wszystkie próby:
	Średnia adekwatność reakcji: 96.0
	Średni Czas reakcji: 770.38
Próby zgodne:
	Średnia adekwatność reakcji: 100.0
	Średni Czas reakcji: 694.8235294117648
Próby niezgodne:
	Średnia adekwatność reakcji: 93.93939393939394
	Średni czas reakcji: 809.3030303030303
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
attention test:
[(959.0416666

In [36]:
file = open("./results/processed_gamers_and_non_gamers_results.csv", "w", encoding="utf-8")
file_gamers_only = open("./results/processed_gamers_results.csv", "w", encoding="utf-8")
file_nonGamers_only = open("./results/processed_non_gamers_results.csv", "w", encoding="utf-8")
delimiter = ";"
example_dict = players[0].test_result
header = []
for key in example_dict:
    if key == "attention_reaction_time_avg" or key == "attention_status_prcnt":
        header.append(key)
        for i in [5, 10, 15, 20]:
            header.append(key + "_{}_dist".format(i))
    elif key == "typology":
        header.extend(typology_column_names)
    elif key == "player_type_values":
        header.extend(player_types_column_names_SV)
    else:
        header.append(key)
files = [file, file_gamers_only, file_nonGamers_only]
for f in files:
    f.write(delimiter.join(header) + "\n")
for player in players:
    player_info = ([player.test_result["id"]])
    for id, result in enumerate(list(player.test_result.values())[1:]):
        if (type(result) is list):
            for res in result:
                player_info.append(str(res).replace(".",","))
        else:
            player_info.append(str(result).replace(".",","))
    player_info = delimiter.join(player_info) + "\n"
    if player.test_result["if_gamer"]:
        file_gamers_only.write(player_info)
    else:
        file_nonGamers_only.write(player_info)
    file.write(player_info)
for f in files:
    f.close()